# Simulation of a Turbo encoder and decoder

In this example we will show how to use the Turbo encoder/decoder class in py-itpp. The Turbo codec used here is quite close to the LTE specification, however, implementation details related to tails have been left out for simplicity. 

In [ ]:
# Imports
import itpp
from matplotlib import pyplot as plt

#  Set up the Turbo encoder/decoder class

In [ ]:
turbo_codec = itpp.comm.turbo_codec()

generator_sequence = itpp.ivec('11, 13')
constraint_length = 4
block_size = 512

# Set coded parameters
turbo_codec.set_parameters(generator_sequence, generator_sequence, constraint_length, itpp.ivec())

# The interleaver is generate based on a Quadrature Permutation Polynomial (QPP)
# Defined in 
f1 = 31
f2 = 64
interleaver_sequence = itpp.ivec(block_size)
for i in range(block_size):
    interleaver_sequence.set(i, (f1 * i + f2 * i * i) % block_size)

turbo_codec.set_interleaver(interleaver_sequence)

# Find out the channel code rate
dummy_bits = itpp.randb(block_size)
coded_bits = itpp.bvec()
turbo_codec.encode(dummy_bits, coded_bits)

coded_block_length = coded_bits.length()
rate = block_size / coded_block_length

print('Channel code rate: %0.4f'%(rate))

# Initialize modulator, channel, and block error counter
In this example we simulate quadrature amplitude modulation (QAM) with 4 constellation points in the complex domain. The channel is additive white gaussian noise (AWGN) normalized to unit power here, and updated in later section for each simulated EbN0.

In [ ]:
# Initialize QAM modulator
qam = itpp.comm.QAM(M=4)

# Initialize an AWGN channel
channel = itpp.comm.AWGN_Channel(noisevar=0)

# Initialize block error counter
blerc = itpp.comm.BLERC(block_size)

# Set up some simulation specific parameters

In [ ]:
EbN0_dB = itpp.vec('0:1:5')
EbN0 = itpp.math.inv_dB(EbN0_dB)

energy_per_bit = 1.0 / rate
noise_variance =  energy_per_bit / EbN0

nrof_blocks = 100 # Number of blocks in each iteration
max_iterations = 10 
max_errors = 10

bler = itpp.vec(EbN0_dB.length()) # Bit error rate
bler.clear()

# Randomize the random number generators
itpp.random.RNG_reset(42)

# Run the simulation

In [ ]:
coded_bits = itpp.bvec()
transmitted_symbols = itpp.cvec()
for p in range(EbN0_dB.length()):
    print('Now simulating point %d out of %d with EbN0_dB = %.2f'%(p + 1, EbN0_dB.length(), EbN0_dB[p]))
    blerc.clear()  # Clear the bit error rate counter.
    channel.set_noise(noise_variance[p]);  # Set the noise value of the AWGN channel.
    for i in range(max_iterations):
      uncoded_bits = itpp.randb(nrof_blocks * block_size) 
      turbo_codec.encode(uncoded_bits, coded_bits)
        
      qam.modulate_bits(coded_bits, transmitted_symbols)
      received_symbols = channel(transmitted_symbols)
    
      soft_values = qam.demodulate_soft_bits(received_symbols, noise_variance[p], itpp.comm.soft_method.LOGMAP)
        
      decoded_bits = itpp.bvec()
      turbo_codec.decode(soft_values, decoded_bits, itpp.bvec())
      blerc.count(uncoded_bits, decoded_bits)
          
      bler[p] = blerc.get_errorrate()
      
      # Break the simulation on this point if sufficient number of bit errors were observed
      if (blerc.get_errors() > max_errors):
        print('Breaking on point %d with %d block errors'%(p + 1, blerc.get_errors()))
        break

# Plot results

In [ ]:
# print(bler)
# print(EbN0_dB)

#Plot the results:
plt.figure()
plt.semilogy(EbN0_dB.to_numpy_ndarray(), bler.to_numpy_ndarray())
plt.xlabel('EbN0_dB')
plt.ylabel('BLER')
plt.show()